In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


/usr/local/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv(r'train.csv', encoding='utf_8_sig')
train.fillna('', inplace=True)
test = pd.read_csv(r'test.csv', encoding='utf_8_sig')
test.fillna('', inplace=True)

In [3]:
train

,label,text,diff
0,negative,Fix #2197 extended regex pattern example,diff --git a/docs/content/manual/manual.yml b/...
1,negative,some test spec fixes,diff --git a/asset-pipeline-grails/src/test/gr...
2,positive,"FILEUPLOAD-279: Introduce a system property, w...",diff --git a/.gitignore b/.gitignore\nnew file...
3,negative,fix(documentation): remove route redeclaration...,diff --git a/doc/API/centreon-api-v2.yaml b/do...
4,negative,[JENKINS-49744] - Generalize the permission ha...,diff --git a/src/main/java/com/synopsys/arc/je...
...,...,...,...
96,negative,[maven-release-plugin] prepare release checkst...,diff --git a/pom.xml b/pom.xml\nindex 2043823....
97,negative,Merge pull request #360 from joaosantos81/mast...,diff --git a/control/Session.php b/control/Ses...
98,negative,Repairing: implement invalid node checksum fix.,diff --git a/libexfat/exfat.h b/libexfat/exfat...
99,positive,[MJ2] Avoid index out of bounds access to pi->...,diff --git a/src/lib/openmj2/pi.c b/src/lib/op...


In [4]:
train_code_change = list(train['diff'].astype(str))
test_code_change = list(test['diff'].astype(str))

In [5]:
# !pip install setfit
from commitfit import get_templated_dataset,sample_dataset
from datasets import Dataset, load_metric

In [6]:
Dataset_train = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

In [7]:
train_dataset = get_templated_dataset(Dataset_train, candidate_labels=['positive','negative'], sample_size=8)

In [8]:
# list(train['message'].astype(str).values)
train_dataset

Dataset({
    features: ['label', 'text', 'diff'],
    num_rows: 117
})

In [9]:
len(train)

101

In [10]:
train['label'].value_counts()

label
negative    63
positive    38
Name: count, dtype: int64

In [11]:
test['label'].value_counts()

label
negative    6284
positive    3727
Name: count, dtype: int64

In [12]:
len(train_dataset)

117

In [13]:
from sklearn import  metrics

def compute_metrics(y_pred, y_test):
    # print(y_pred,y_test)
    # classification_report = metrics.classification_report(y_test,y_pred)
    accuracy_score = metrics.accuracy_score(y_test,y_pred)
    precision_score = metrics.precision_score(y_test,y_pred, average='weighted')
    recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
    f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

    # return {"classification_report": classification_report}
    return {"accuracy": accuracy_score, "precision": precision_score,"recall": recall_score, "f1":f1_score}

In [14]:
test['label'].value_counts()

label
negative    6284
positive    3727
Name: count, dtype: int64

In [15]:
train_dataset

Dataset({
    features: ['label', 'text', 'diff'],
    num_rows: 117
})

In [16]:
train

,label,text,diff
0,negative,Fix #2197 extended regex pattern example,diff --git a/docs/content/manual/manual.yml b/...
1,negative,some test spec fixes,diff --git a/asset-pipeline-grails/src/test/gr...
2,positive,"FILEUPLOAD-279: Introduce a system property, w...",diff --git a/.gitignore b/.gitignore\nnew file...
3,negative,fix(documentation): remove route redeclaration...,diff --git a/doc/API/centreon-api-v2.yaml b/do...
4,negative,[JENKINS-49744] - Generalize the permission ha...,diff --git a/src/main/java/com/synopsys/arc/je...
...,...,...,...
96,negative,[maven-release-plugin] prepare release checkst...,diff --git a/pom.xml b/pom.xml\nindex 2043823....
97,negative,Merge pull request #360 from joaosantos81/mast...,diff --git a/control/Session.php b/control/Ses...
98,negative,Repairing: implement invalid node checksum fix.,diff --git a/libexfat/exfat.h b/libexfat/exfat...
99,positive,[MJ2] Avoid index out of bounds access to pi->...,diff --git a/src/lib/openmj2/pi.c b/src/lib/op...


In [17]:
model_id = "sentence-transformers/paraphrase-mpnet-base-v2"

In [18]:
from commitfit import CommitFitModel
model = CommitFitModel.from_pretrained(model_id)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [19]:
from commitfit import CommitFitTrainer

trainer = CommitFitTrainer(
    model=model,
    train_dataset=train_dataset,
    train_code_change = train_code_change,
    test_code_change = test_code_change,
    eval_dataset=test_dataset,
    metric = compute_metrics,
    num_iterations=20,
    num_epochs=1
)

In [20]:
%%time
trainer.train()
fewshot_metrics = trainer.evaluate()
fewshot_metrics

Generating Training Pairs: 100%|██████████| 20/20 [00:00<00:00, 114.72it/s]
***** Running training *****
  Num examples = 4680
  Num epochs = 1
  Total optimization steps = 293
  Total train batch size = 16
Epoch: 100%|██████████| 1/1 [01:07<00:00, 67.42s/it]
***** Running evaluation *****


CPU times: user 14min 32s, sys: 1min 8s, total: 15min 41s
Wall time: 4min 42s


{'accuracy': 0.8001198681450404,
 'precision': 0.8011308391660051,
 'recall': 0.8001198681450404,
 'f1': 0.8005661449401407}